In [56]:
#!/home/tbarnhart/miniconda3/envs/py36/bin python
# script to extract watershed areas from gauges snapped to NHDplusV2.1
# Theodore Barnhart | August 22, 2018 | tbarnhart@usgs.gov

#from grass.pygrass.modules import Module
import sys
import subprocess
import os
import pandas as pd
from pyproj import Proj, transform
import numpy as np

In [18]:
#reg = sys.argv[1] # extract coommand line argument 1, region to process
reg = '16'

In [ ]:
workspace = '/home/tbarnhart/projects/DEM_processing/data'
drainDirPath = os.path.join(workspace,'NHDplusV21_facfdr','region_%s_fdr.tiff'%(reg))

In [39]:
fl = '/data1/projectsx/WAUSP_National_Model/CPG_data/CATCHMENT_v1.csv'
gauges = pd.read_csv(fl)

In [40]:
gauges.loc[gauges.Lat_snap == -9999,'Lat_snap'] = np.NaN
gauges.loc[gauges.Long_snap == -9999,'Long_snap'] = np.NaN

In [41]:
def fixRegion(df):
    '''Fix region 3, which is divided in the original data set for some reason.'''
    reg = df.NHDPlusReg # get region
    
    if reg[:2] == '03':
        reg = '03'
        
    return reg

In [42]:
gauges['NHDPlusReg'] = gauges.apply(fixRegion,axis=1) # fix region 3

In [43]:
gauges = gauges.loc[gauges.NHDPlusReg == reg]

In [77]:
# create list of gauge locations

# infill the missing snapped locations with NWIS locations
gauges.loc[np.isnan(gauges.Lat_snap) == 1,'Lat_snap'] = gauges.loc[np.isnan(gauges.Lat_snap) == 1].Lat_nwis
gauges.loc[np.isnan(gauges.Long_snap) == 1,'Long_snap'] = gauges.loc[np.isnan(gauges.Long_snap) == 1].Long_nwis

lons = gauges.Long_snap.values
lats = gauges.Lat_snap.values
gaugeID = gauges.Gage_no.values

In [68]:
inProj = '+proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs' # projection of points pulled from shapefile
outProj = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs' # pulled from FDR

inProj = Proj(inProj)
outProj = Proj(outProj)

In [74]:
# reporject the gauges
xs = []
ys = []

for lon,lat in zip(lons,lats):
    x,y = transform(inProj,outProj,lon,lat)
    xs.append(x)
    ys.append(y)

In [ ]:
# now for the GRASS code

# define functions to use
r_external = Module('r.external')
r_water_outlet = Module('r.water.outlet')
r_to_vect = Module('r.to.vect')
v_out_ogr = Module('v.out.ogr')
g_region = Module('g.region')

In [ ]:
r_external(input=drainDirPath,output='dir')
r_region(res = 30) # set region resolution
for ID,x,y in zip(gaugeID,xs,ys):
    outfl = os.path.join(workspace,'gauges','region_%s_gageNo_%s_watershed_NHDplusV21.shp'%(reg,ID)) # format output string
    r_water_outlet(input = 'dir', output = 'watershed', coordinates = '%s,%s'%(x,y), overwrite=True) # delineate watershed
    r_to_vect(input = 'watershed', output = 'boundary', type = 'area') # convert raster to vector
    v_out_ogr(input = 'boundary', type = 'area', output = 'outfl', format = 'ESRI_Shapefile') # export the watershed boundary